In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
from algorithm.kgvec2go_vectors_simple import run_method, load_gensim_model
from excludes.basic_excludes import (node_excl_yago_func, node_excl_extra,
                                     node_excl_owlthing_func, node_excl_wiki_func)
from loaders.loaders import *
from evaluation.split_dataset import split_on_seed_dataset
import json
import pickle
import os
from tqdm import tqdm

exclude_paths = [
    ["", "http://www.w3.org/1999/02/22-rdf-syntax-ns#type", "http://www.w3.org/2004/02/skos/core#Concept"]
]
relation_excludes = ["http://xmlns.com/foaf/0.1/primaryTopic",
                     "http://dbpedia.org/property/isCitedBy",
                     "http://dbpedia.org/ontology/wikiPageWikiLink",
                     "http://dbpedia.org/ontology/wikiPageWikiLinkText",
                     "http://dbpedia.org/property/wikiPageUsesTemplate",
                     'http://dbpedia.org/ontology/wikiPageRedirects',
                     'http://dbpedia.org/ontology/wikiPageOutDegree',
                     "http://dbpedia.org/ontology/abstract",
                     "http://www.w3.org/2000/01/rdf-schema#comment", ]

res_excludes_inside_point_relation = ["http://purl.org/linguistics/gold/hypernym",
                                      "http://www.w3.org/1999/02/22-rdf-syntax-ns#type"]
res_excludes_inside_point_node = []
node_excludes_funcs = [node_excl_wiki_func, node_excl_owlthing_func, node_excl_yago_func, node_excl_extra]

SEED = 44
SAMPLE_SIZE = 4
verbose = 1

#Dataset load
#lc_quad_train = load_lc_quad_train()

lc_quad_train = load_on_path(ds_lc_quad_train_cleaned)
splitted_lcquad = split_on_seed_dataset(lc_quad_train, SAMPLE_SIZE, SEED)

#splitted_lcquad = load_on_path_pickle("/home/kardosp/continuethelist/outputs/graphwalk_v2_output_sample4.pickle")

cwd = "/home/kardosp/continuethelist/dbpedia_vectors/kgvec2go"
distance_metric = "l1ellipse"
assert distance_metric in ["ellipse", "circle", "l1ellipse", "euclidean"]

word2vec_path = os.path.join(cwd, 'model.kv')
wv_model = load_gensim_model(word2vec_path)
missing_counter = 0

In [ ]:
# lcquad_output = run_on_dataset(splitted_lcquad, exclude_paths, relation_excludes, node_excludes_funcs)
for i, record in tqdm(list(enumerate(splitted_lcquad))):
    print(i)
    if "output" in record:
        continue
    result, missing = run_method(record["seed"], wv_model, distance=distance_metric)
    record["output"] = result
    missing_counter += missing
    print(missing_counter)
    # TODO - evaluate


In [11]:
from gensim.models import Word2Vec, KeyedVectors
from gensim import models
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.neighbors import RadiusNeighborsClassifier
from copy import copy


def load_gensim_model(path):
    w2v_model = models.KeyedVectors.load(
        path, mmap="r"
    )
    return w2v_model


def ellipse_rectangle_dist(center, radius, point, eps=1e-4):
    res = np.max(np.power(np.divide(np.subtract(center, point), radius), 2))
    # res = np.sum(np.power(np.divide(np.subtract(center, point), radius), 2))
    return res


def get_radius(center, seed):
    return np.max(np.abs(np.array(seed) - np.array(center)), axis=0)


def run_method(seed, wv_model: KeyedVectors, distance="circle"):
    """
    Input is a single record's seed.
    distance can be the following: ["circle", "euclidean", "ellipse", "l1ellipse"]
    :return:
    """

    vectors = dict()
    missing = 0
    for item in seed:
        try:
            vectors[item] = list(wv_model[item])
        except Exception as e:
            missing += 1
    if missing == len(seed):
        return None, missing
    center_vector = np.mean(list(vectors.values()), axis=0)
    closer_elements = list()

    if distance == "circle" or distance == "euclidean":
        point_dist = cdist([center_vector], list(vectors.values()), metric="euclidean")
        threshold = np.max(point_dist)
        eps = 1e-5
        all_euclidean_dist = cdist([center_vector], wv_model.vectors, metric="euclidean")
        for index in all_euclidean_dist.argsort()[0]:
            if all_euclidean_dist[0][index] > threshold:
                break
            closer_elements.append((index, all_euclidean_dist[0][index]))
    elif distance == "ellipse":
        radius = get_radius(center_vector, list(vectors.values()))
        center_vector /= radius
        wv_vectors = copy(wv_model.vectors)
        wv_vectors /= radius

        point_vectors = [v/radius for v in vectors.values()]
        point_dist = cdist([center_vector], point_vectors, metric="euclidean")
        threshold = np.max(point_dist)
        all_euclidean_dist = cdist([center_vector], wv_vectors, metric="euclidean")
        for index in all_euclidean_dist.argsort()[0]:
            if all_euclidean_dist[0][index] > threshold:
                break
            closer_elements.append((index, all_euclidean_dist[0][index]))

    elif distance == "l1ellipse":
        radius = get_radius(center_vector, list(vectors.values()))
        all_l1ellipse_dist = cdist([center_vector], wv_model.vectors, metric=lambda x, y: ellipse_rectangle_dist(x, radius, y))
        all_l1ellipse_dist_1d = all_l1ellipse_dist[0]
        sorted_distances = all_l1ellipse_dist_1d.argsort()

        for index in sorted_distances:
            if all_l1ellipse_dist_1d[index] <= 1:
                closer_elements.append((index, all_l1ellipse_dist_1d[index]))
            else:
                break
    prediction = [wv_model.index_to_key[item[0]] for item in closer_elements]
    return prediction, missing

# distance_result = cdist([center_vector], [wv_model.vectors], metric=lambda x, y: is_inside_ellipse(x, radius, y))


In [12]:
import numpy as np

In [13]:
record = splitted_lcquad[3]
seed = record["seed"]
distance=distance_metric

vectors = dict()
missing = 0
for item in seed:
    try:
        vectors[item] = list(wv_model[item])
    except Exception as e:
        missing += 1
if missing == len(seed):
    print("END END END END")
center_vector = np.mean(list(vectors.values()), axis=0)
closer_elements = list()

radius = get_radius(center_vector, list(vectors.values()))
all_l1ellipse_dist = cdist([center_vector], wv_model.vectors, metric=lambda x, y: ellipse_rectangle_dist(x, radius, y))
all_l1ellipse_dist_1d = all_l1ellipse_dist[0]
sorted_distances = all_l1ellipse_dist_1d.argsort()

for index in sorted_distances:
    if all_l1ellipse_dist_1d[index] <= 1:
        closer_elements.append((index, all_l1ellipse_dist_1d[index]))
    else:
        break
prediction = [wv_model.index_to_key[item[0]] for item in closer_elements]

In [14]:
radius

array([0.11844432, 0.97818696, 0.07799672, 0.8200105 , 0.22999752,
       0.21914172, 0.01801936, 0.36768246, 0.25332588, 0.29013836,
       0.20524928, 0.3717527 , 0.4937508 , 0.14497097, 0.2286574 ,
       0.02229915, 0.07086751, 0.54856634, 0.5900674 , 0.20265542,
       0.36240995, 0.18179011, 0.06202868, 0.03743804, 0.34904706,
       0.00381739, 0.1498371 , 0.3766067 , 0.20934474, 0.40434203,
       0.42506683, 0.19579357, 0.760578  , 0.32830268, 0.49670887,
       0.22586542, 0.00953475, 0.2868684 , 0.61026686, 0.7729161 ,
       0.03186053, 0.2010938 , 0.00843189, 0.44407126, 0.10356328,
       0.01474166, 0.06465513, 0.31005275, 0.7670278 , 0.09526935,
       0.35222226, 0.02939075, 0.0296835 , 0.3366586 , 0.15370667,
       0.55364144, 0.24478333, 0.36557266, 0.4063843 , 0.08941403,
       0.2850541 , 0.2321742 , 0.19295101, 0.37854952, 0.03865576,
       0.2946167 , 0.44002187, 0.14210726, 0.6190351 , 0.20702215,
       0.09911925, 0.5432849 , 0.35490376, 0.34426785, 0.00417

In [15]:
all_l1ellipse_dist

array([[17685.09570312, 16285.28808594, 33932.421875  , ...,
        19162.18359375, 17290.96679688, 20069.09570312]])

In [ ]:
len(vectors)

In [16]:
for item in vectors.values():
    print(ellipse_rectangle_dist(center_vector, radius, item))

1.0
1.0
